<img src="Tarjeta.png">

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Antes-de-empezar" data-toc-modified-id="Antes-de-empezar-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Antes de empezar</a></span></li><li><span><a href="#Conectar-área-de-trabajo" data-toc-modified-id="Conectar-área-de-trabajo-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Conectar área de trabajo</a></span></li><li><span><a href="#Crear-un-dataset-base" data-toc-modified-id="Crear-un-dataset-base-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Crear un dataset base</a></span></li><li><span><a href="#Crear-un-dataset-target" data-toc-modified-id="Crear-un-dataset-target-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Crear un dataset <em>target</em></a></span></li><li><span><a href="#Crear-el-seguimiento-del-data-drift" data-toc-modified-id="Crear-el-seguimiento-del-data-drift-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Crear el seguimiento del data drift</a></span><ul class="toc-item"><li><span><a href="#Crear-un-cluster-de-proceso" data-toc-modified-id="Crear-un-cluster-de-proceso-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>Crear un cluster de proceso</a></span></li><li><span><a href="#Definir-el-seguimiento-del-data-drift" data-toc-modified-id="Definir-el-seguimiento-del-data-drift-5.2"><span class="toc-item-num">5.2&nbsp;&nbsp;</span>Definir el seguimiento del data drift</a></span></li></ul></li><li><span><a href="#Rellenar-el-seguimiento-del-data-drift" data-toc-modified-id="Rellenar-el-seguimiento-del-data-drift-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Rellenar el seguimiento del data drift</a></span></li><li><span><a href="#Analizar-el-data-drift" data-toc-modified-id="Analizar-el-data-drift-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Analizar el data drift</a></span></li><li><span><a href="#Más-información" data-toc-modified-id="Más-información-8"><span class="toc-item-num">8&nbsp;&nbsp;</span>Más información</a></span></li></ul></div>

# Monitoring Data Drift

Cuando pasa el tiempo, el modelo puede perder su efectividad y su poder predictivo. La principal causa es el cambio de la tendencia de las variables, a esto se le llama *data drift* y es importante medirlo y hacer seguimiento del mismo para detectar si es necesario el reentrenamiento del modelo.

## Antes de empezar

Instalamos el módulo de datadrift de azure ml

In [1]:
!pip show azureml-datadrift

Name: azureml-datadrift
Version: 1.32.0
Summary: Azure Machine Learning datadrift
Home-page: https://docs.microsoft.com/python/api/overview/azure/ml/?view=azure-ml-py
Author: Microsoft Corp
Author-email: None
License: https://aka.ms/azureml-sdk-license
Location: /anaconda/envs/azureml_py36/lib/python3.6/site-packages
Requires: msrest, pyspark, jsonpickle, numpy, azureml-dataset-runtime, azureml-core, azureml-pipeline-core, scipy, scikit-learn, lightgbm, matplotlib, pandas, azureml-telemetry
Required-by: 


## Conectar área de trabajo

In [2]:
import azureml.core
from azureml.core import Workspace

# Cargar el área de trabajo del fichero de configuración
ws = Workspace.from_config()
print('Versión de Azure ML {} y área de trabajo {}'.format(azureml.core.VERSION, ws.name))

Versión de Azure ML 1.32.0 y área de trabajo aml-nuclio


## Crear un dataset base

Para monitorear el data drift se necesita tener un dataset base (normalmente es con el que se entrena el modelo) para usarlo como punto de comparación en el futuro.

In [19]:
from azureml.core import Datastore, Dataset


# Subir dataset base
default_ds = ws.get_default_datastore()
default_ds.upload_files(files=['./data/winequality.csv'],
                       target_path='vino-baseline',
                       overwrite=True, 
                       show_progress=True)

# Crear y registrar el dataset base
print('Registrando dataset base...')
baseline_data_set = Dataset.Tabular.from_delimited_files(path=(default_ds, 'vino-baseline/*.csv'), separator=';')
baseline_data_set = baseline_data_set.register(workspace=ws, 
                           name='vino baseline',
                           description='vino baseline',
                           tags = {'format':'CSV'},
                           create_new_version=True)

print('Dataset base registrado!')

Uploading an estimated of 1 files
Uploading ./data/winequality.csv
Uploaded ./data/winequality.csv, 1 files out of an estimated total of 1
Uploaded 1 files
Registrando dataset base...
Dataset base registrado!


## Crear un dataset *target*


Con el tiempo, puedes recoger nuevos datos con las mismas variables de tus datos de entrenamiento. Para comparar estos nuevos datos con tus datos base, se debe de definir un dataset target que incluye las variables que tu quieres analizar para medir el data drift.

In [6]:
import pandas as pd
data = pd.read_csv('data/winequality.csv', sep=';', decimal='.')
data.head()

,fixed_acidity,volatile_acidity,citric_acid,residual_sugar,chlorides,free_sulfur_dioxide,total_sulfur_dioxide,density,ph,sulphates,alcohol,top_quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,0
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,0
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,0
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,0
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,0


In [11]:
file_paths

['data/winequality_2021-08-02.csv',
 'data/winequality_2021-08-09.csv',
 'data/winequality_2021-08-16.csv',
 'data/winequality_2021-08-23.csv',
 'data/winequality_2021-08-30.csv',
 'data/winequality_2021-09-06.csv']

In [13]:
import datetime as dt
import pandas as pd

print('Generando dato simulado...')

data = pd.read_csv('data/winequality.csv', sep=';', decimal='.')
data["alcohol"] = pd.to_numeric(data["alcohol"], errors='coerce')
data = data.dropna()

# Generamos datos pasados 6 semanas
weeknos = reversed(range(6))

file_paths = []
for weekno in weeknos:
    
    data_date = dt.date.today() - dt.timedelta(weeks=weekno)
    
    # Modificamos los datos para crear drift
    data['volatile_acidity'] = data['volatile_acidity'] + 1
    data['total_sulfur_dioxide'] = round(data['total_sulfur_dioxide'] * 1.2).astype(int)
    data['chlorides'] = data['chlorides'] * 1.1
    
    # Guardamos los datos con la fecha de la semana
    file_path = 'data/vino_{}.csv'.format(data_date.strftime("%Y-%m-%d"))
    data.to_csv(file_path)
    file_paths.append(file_path)

# Subimos los ficheros
path_on_datastore = 'vino-target'
default_ds.upload_files(files=file_paths,
                       target_path=path_on_datastore,
                       overwrite=True,
                       show_progress=True)

# Carpeta de las particiones
partition_format = path_on_datastore + '/vino_{date:yyyy-MM-dd}.csv'
target_data_set = Dataset.Tabular.from_delimited_files(path=(default_ds, path_on_datastore + '/*.csv'),
                                                       partition_format=partition_format)

# Register the target dataset
print('Registrando dataset target...')
target_data_set = target_data_set.with_timestamp_columns('date').register(workspace=ws,
                                                                          name='vino target',
                                                                          description='vino target data',
                                                                          tags = {'format':'CSV'},
                                                                          create_new_version=True)

print('Dataset target registrado!')

Generando dato simulado...
Uploading an estimated of 6 files
Uploading data/vino_2021-08-02.csv
Uploaded data/vino_2021-08-02.csv, 1 files out of an estimated total of 6
Uploading data/vino_2021-08-09.csv
Uploaded data/vino_2021-08-09.csv, 2 files out of an estimated total of 6
Uploading data/vino_2021-08-16.csv
Uploaded data/vino_2021-08-16.csv, 3 files out of an estimated total of 6
Uploading data/vino_2021-08-23.csv
Uploaded data/vino_2021-08-23.csv, 4 files out of an estimated total of 6
Uploading data/vino_2021-08-30.csv
Uploaded data/vino_2021-08-30.csv, 5 files out of an estimated total of 6
Uploading data/vino_2021-09-06.csv
Uploaded data/vino_2021-09-06.csv, 6 files out of an estimated total of 6
Uploaded 6 files
Registrando dataset target...
Dataset target registrado!


## Crear el seguimiento del data drift

El seguimiento del data drift se ejecutará periodicamente o a demanda para comparar con el dataset base.

### Crear un cluster de proceso

In [14]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

cluster_name = "NuclioCluster"

try:
    # Validar si existe
    inference_cluster = ComputeTarget(workspace=ws, name=cluster_name)
    print('Cluster de proceso encontrado.')
except ComputeTargetException:
    # Si no existe, se crea
    try:
        compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS11_V2', max_nodes=2)
        inference_cluster = ComputeTarget.create(ws, cluster_name, compute_config)
        inference_cluster.wait_for_completion(show_output=True)
    except Exception as ex:
        print(ex)
    

Cluster de proceso encontrado.


### Definir el seguimiento del data drift

Se usará la clase **DataDriftDetector*.

In [15]:
from azureml.datadrift import DataDriftDetector

# Variables
features = ['volatile_acidity', 'total_sulfur_dioxide', 'chlorides']

# Detección data drift
monitor = DataDriftDetector.create_from_datasets(ws, 'nuclio-vino-drift', baseline_data_set, target_data_set,
                                                      compute_target=cluster_name, 
                                                      frequency='Week', 
                                                      feature_list=features, 
                                                      drift_threshold=.3, 
                                                      latency=24)
monitor

{'_workspace': Workspace.create(name='aml-nuclio', subscription_id='87f9793d-5515-43eb-b182-0f27b97da8b3', resource_group='nuclio'), '_frequency': 'Week', '_schedule_start': None, '_schedule_id': None, '_interval': 1, '_state': 'Disabled', '_alert_config': None, '_type': 'DatasetBased', '_id': '5e490dca-1126-4e1d-89d6-9bd61c84a922', '_model_name': None, '_model_version': 0, '_services': None, '_compute_target_name': 'NuclioCluster', '_drift_threshold': 0.3, '_baseline_dataset_id': 'a2ee43e3-5a63-4faa-af04-444283eebbe0', '_target_dataset_id': '062b2350-4aba-493f-9392-c901f4044343', '_feature_list': ['volatile_acidity', 'total_sulfur_dioxide', 'chlorides'], '_latency': 24, '_name': 'nuclio-vino-drift', '_latest_run_time': None, '_client': <azureml.datadrift._restclient.datadrift_client.DataDriftClient object at 0x7f63d3c26908>, '_logger': <_TelemetryLoggerContextAdapter azureml.datadrift._logging._telemetry_logger.azureml.datadrift.datadriftdetector (DEBUG)>}

## Rellenar el seguimiento del data drift

Se tiene un dataset base y un dataset target que incluye las simulaciones de 6 semanas de datos recogidas.

In [ ]:
from azureml.widgets import RunDetails

backfill = monitor.backfill(dt.datetime.now() - dt.timedelta(weeks=6), dt.datetime.now())

RunDetails(backfill).show()
backfill.wait_for_completion()

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

## Analizar el data drift

In [ ]:
drift_metrics = backfill.get_metrics()
for metric in drift_metrics:
    print(metric, drift_metrics[metric])

## Más información

[Detección de datadrift](https://docs.microsoft.com/azure/machine-learning/how-to-monitor-datasets)

[Recopilación de datos de modelos en producción](https://docs.microsoft.com/azure/machine-learning/how-to-enable-data-collection) 